::::
:::{thebe-button}
:::
::::

# Get thermal data


In [ ]:
from functools import cache

from boilercore.fits import fit_from_params
from boilercore.models.geometry import GEOMETRY
from boilercore.paths import ISOLIKE, dt_fromisolike
from devtools import pprint
from matplotlib.pyplot import subplots
from pandas import DataFrame, Series, concat, read_csv, read_hdf
from seaborn import lineplot, scatterplot

from boilercv_docs.nbs import display_markdown, init
from boilercv_pipeline.models.column import Col
from boilercv_pipeline.stages.find_objects import FindObjects
from boilercv_pipeline.stages.get_thermal_data import GetThermalData as Params

PARAMS = None

In [ ]:
if isinstance(PARAMS, str):
    params = Params.model_validate_json(PARAMS)
else:
    params = Params(context=init(), load_src_from_outs=True)

params.format.set_display_options()
data = params.data
C = params.cols
context = params.context


def preview(
    df: DataFrame, cols: list[Col] = C.dests, index: Col = C.time_elapsed, head: int = 5
) -> DataFrame:
    """Preview dataframes in this notebook."""
    display_markdown(
        df.drop(columns=C.time())
        .set_index(index())[[c() for c in cols if c not in [C.time, index]]]
        .head(head),
        floatfmt=params.format.floatfmt,
    )
    return df


# ? Avoids recomputing in tight loop below
MODELS = params.fit.get_models(params.deps.modelfunctions)[0]


@cache
def fit(ser: tuple[float, ...]) -> tuple[dict[str, float], dict[str, float]]:
    """Fit for each set of temperatures."""
    return fit_from_params(MODELS, params.fit, GEOMETRY.rods["R"], ser)


def apply_fit(df: DataFrame) -> "Series[float]":
    """Fit model function across sample temperatures."""
    return df.loc[:, [c() for c in C.sample_temps]].apply(
        lambda ser: fit(tuple(ser))[0]["q_s"] * C.flux.scale, axis="columns"
    )


pprint(params)

In [ ]:
sources = [c.src for c in C.sources]
data.dfs.src = preview(
    cols=[c for c in C.sources if c not in C.sample_temps[2:]],
    df=concat([
        read_csv(
            p,
            usecols=[C.time.src, *sources],
            parse_dates=[C.time.src],
            index_col=C.time.src,
        )
        for p in params.deps.thermal_paths
    ])
    .reset_index()
    .rename(columns={c.src: c() for c in C.sources})
    .assign(**{
        C.time_elapsed(): lambda df: (
            (df[C.time()] - df[C.time()][0]).dt.total_seconds()
        )
    }),
)

In [ ]:
only_dests = [c for c in C.dests if c not in [*C.sources, C.boiling]]
data.dfs.dst = preview(
    cols=only_dests,
    df=(
        DataFrame(read_hdf(params.outs.df))
        if params.load_src_from_outs and params.outs.df.exists()
        else data.dfs.src.set_index(C.time())
        .resample("s")
        .median()
        .assign(**{
            C.video(): lambda df: df.index.isin(
                df.index[
                    df.index.get_indexer(
                        [
                            dt_fromisolike(match)
                            for p in FindObjects(context=context).contours
                            if (match := ISOLIKE.search(p.stem))
                        ],
                        method="nearest",
                    )
                ]
            )
        })
        .reset_index()
        .ffill()
        .assign(**{
            C.time_elapsed(): lambda df: (
                (df[C.time()] - df[C.time()][0]).dt.total_seconds()
                * C.time_elapsed.scale
            ),
            C.time_elapsed_min(): lambda df: (
                df[C.time_elapsed()] * C.time_elapsed_min.scale
            ),
            C.water_temp(): lambda df: df[[c() for c in C.water_temps]].mean(
                axis="columns"
            ),
            C.boiling(): lambda df: df[C.water_temp()].max(),
            C.superheat(): lambda df: df[C.surface_temp()] - df[C.boiling()],
            C.subcool(): lambda df: df[C.boiling()] - df[C.water_temp()],
            C.flux(): apply_fit,
        })
    )[[c() for c in C.dests]],
)

In [ ]:
data.dfs.resampled = preview(
    index=C.time_elapsed,
    cols=only_dests,
    df=(
        data.dfs.dst.set_index(C.time())
        .resample("20s")
        .agg({
            **dict.fromkeys([c() for c in C.dests if c not in C.idx], "median"),
            C.video(): "max",
        })
        .reset_index()
    ),
)

In [ ]:
data.plots.subcool_superheat, ax = subplots()
lineplot(
    ax=ax,
    zorder=1,
    data=data.dfs.resampled.set_index(C.time_elapsed_min())[
        [C.subcool(), C.superheat()]
    ],
    dashes=False,
    errorbar=None,
)
scatterplot(
    ax=ax,
    s=10 * params.format.marker_scale,
    markers={C.video(): "*"},
    palette={C.video(): "red"},
    data=data.dfs.resampled.assign(**{
        C.video(): lambda df: df[df[C.video()]][C.superheat()]
    }).set_index(C.time_elapsed_min())[[C.video()]],
)
scatterplot(
    ax=ax,
    s=10 * params.format.marker_scale,
    markers={C.video(): "*"},
    palette={C.video(): "red"},
    data=data.dfs.resampled.assign(**{
        C.video(): lambda df: df[df[C.video()]][C.subcool()]
    }).set_index(C.time_elapsed_min())[[C.video()]],
    legend=False,
)
ax.set_ylabel(C.subcool.ylabel)
params.format.move_legend(ax)

In [ ]:
data.plots.subcool, ax = subplots()
scatterplot(
    ax=ax, data=data.dfs.resampled, x=C.subcool(), y=C.flux(), hue=C.time_elapsed_min()
)
scatterplot(
    ax=ax,
    s=10 * params.format.marker_scale,
    markers={C.video(): "*"},
    palette={C.video(): "red"},
    data=data.dfs.resampled.assign(**{
        C.video(): lambda df: df[df[C.video()]][C.flux()]
    }).set_index(C.subcool())[[C.video()]],
)
ax.get_legend().set_title(C.time_elapsed_min())  # pyright: ignore[reportOptionalMemberAccess]
params.format.move_legend(ax, ncol=4)

In [ ]:
data.plots.superheat, ax = subplots()
scatterplot(
    ax=ax,
    data=data.dfs.resampled,
    x=C.superheat(),
    y=C.flux(),
    hue=C.time_elapsed_min(),
)
scatterplot(
    ax=ax,
    s=10 * params.format.marker_scale,
    markers={C.video(): "*"},
    palette={C.video(): "red"},
    data=data.dfs.resampled.assign(**{
        C.video(): lambda df: df[df[C.video()]][C.flux()]
    }).set_index(C.superheat())[[C.video()]],
)
ax.get_legend().set_title(C.time_elapsed_min())  # pyright: ignore[reportOptionalMemberAccess]
params.format.move_legend(ax, ncol=4)